# Humidity Sensor Project
Author: Stefan Roland Schwingenschlögl <br>
email: stefan.roland.schwingenschloegl@gmail.com <br>
github: github.com/stefan-schwingenschloegl <br>
___
*Projekt File No: 1 <br>*

In [ ]:
import serial
import time
import numpy as np
import pyodbc
import datetime

In [ ]:
addr = "COM6" ## serial port to read data from
baud = 9600 ## baud rate for instrument

ser = serial.Serial(port = addr, baudrate = baud, timeout=None)

## Database Creation

In [ ]:
# set properties for database
db_name = 'ArduinoDB'
server = 'DESKTOP-JV1HTQR\SQLEXPRESS'
db_connection = False

In [ ]:
# establish DB Connection
def db_connect(server, db_name):
    conn = pyodbc.connect("driver={SQL Server};server="+server+"; database="+db_name+"; trusted_connection=true")
    db_connection = True
    #print(f"\nConnection with {server} sucessfull!\n"
     #     f"Current Database: {db_name}\n"
      #    f"DB Connection Status: {db_connection}")
    return conn

In [ ]:
# close DB Connection
def close_connection(server, conn):
    conn.close()
    db_connection = False
   # print(f"\nDB-Connection with Server {server} closed.\n"
     #     f"DB Connections Status: {db_connection}")

In [ ]:
# create Arduino Database if it does not exist
def create_DB(server, db_name):
    
    # establish connection to master database
    conn = db_connect(server=server, db_name='master')
    
    with conn:
        crs=conn.cursor()
        crs.execute(f"IF NOT EXISTS(SELECT * FROM sys.databases WHERE name = '{db_name}') CREATE DATABASE {db_name};")   
        print(f"{db_name} sucessfully created.")
    close_connection(server=server, conn=conn)

In [ ]:
# create ArduinoDB
create_DB(server, db_name)

## Table Creation

In [ ]:
# function to create table schema for static file
def create_table(table_name, sql_command, server, db_name):
    """
    Create table in relational Database
    
    Input: string: name of table, 
           string: sql-command which should be executed
           string: name of database server
           string: name of database name in database server
    Output: None
    """
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        crs.execute(sql_command)
        print(f"\nTable '{table_name}' available in {db_name}")
    close_connection(server = server, conn=conn)

In [ ]:
sql_command = '''
IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='sensor_values' AND table_schema='dbo') 
                 CREATE TABLE sensor_values (
                        [timestamp] DATETIME2,
                        [hum] numeric(4,2),
                        [temp] numeric(4,2)
                        )
'''
create_table('sensor_values', sql_command, server, db_name)

## Get Values and insert into Database

In [ ]:
def write_value(values):
    """
    Insert values in format "timestamp, humidity, temperature" into Database
    
    Input: list ['humidity', 'temperature']
    Output: None
    """
    conn = db_connect(server=server, db_name=db_name)
    time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # get current timestamp
    
    if values == ['0']: # check if values are '0' -> if yes overwrite with Null
        values = [None, None]
        print(f"Missing Values at {time}!")

    with conn:
        crs=conn.cursor()
        crs.execute("INSERT INTO [dbo].[sensor_values] ([timestamp], [hum], [temp]) VALUES( ?, ?, ?)", time, values[0], values[1])
    close_connection(server = server, conn = conn)
    print(f"Hum: {values[0]}, Temp: {values[1]}", end='\r')

In [ ]:
# while loop to get data over serial and insert values into database
# wait for 30 seconds for next entry
# for now data gets collected via infity loop - termination manually
while True:
    value = ser.readline()
    values = value.decode("utf-8")[:-2].split('T') # split values at seperator 'T'
    write_value(values)
    bufClear = ser.read(ser.inWaiting())
    time.sleep(30)